# Introduction & Background

In this project, we will model the trusted Mauna Loa Observatory data set, a timeseries of weekly atmospheric C02 concentration data collected since 1958. 

![Mauna Loa Data](/Users/andrevacha/Desktop/cs146/Final Project/base_plot.png "Mauna Loa Data")

This figure, known as the Keeling curve, is far from innocuous: it suggests a rising trend in CO2 decoupled from the rhythms of seasonality, and is the quantitative basis for anthropogenic climate change. 

To build modeling intuitions, we will first decompose the data into a series of smaller components. 

## Exploratory Analysis

To make the timeseries more amenable to analysis, we perform a simple transformation: the timestamps are converted to a continuous time 'Years' variable, measuring the number of years since 1958. We can now examine the data piecewise.

Detrended with a savgol filter and anchored against a simple least-squares linear fit, the data seems to follow a non-linear growth pattern. The characteristic bow shape of exponential growth seems to be suggested, though there is a degree of medium-term variation also at play. In the interest of simplicity, we will neglect a potential medium-term component. 

Netting out the long-term component, we can see a strongly periodic seasonal component. Traced with sine-wave (blue) oscillations, it appears to be sinusoidal with a period of 1 year (see annotated 'T') and amplitute of about 3 ppm. These parametric insights will come in handy when we define hyperiors in a later section. This suffices as a characteristic time scale because there is an underlying annual carbon fertilization cycle in terrestrial plants: in winter, lower insolation triggers CO2 increases, and in summer, higher insolation triggers CO2 decreases. 

Finally, we extract a fine-grained noise component. In modeling noise, we need to check that noise is uncorrelated with other observed variables.

The noise is strikingly homoscedastic across the time series, and Gaussian-distributed along the C02 marginal with mean 0. In other words, this is a kind of idealized white noise: ocean winds creating a flux in CO2 readings, light day-to-day miscalibrations, and so on.

# Modeling

We will construct two models: a linear-cosine model and an exponential-cosine model. The former can be thought of as the counterfactual or "middle child" model, while the latter can be thought of as the improved model. We will not go into Bayes Factor analysis, but the limitations of the first model will be discussed in its section. 

## Linear-Cosine Model

![Factor Graph](/Users/andrevacha/Desktop/cs146/Final Project/factor_graph.png "Factor Graph")

The time variable $t$ and CO2 variable are observed (shaded in grey), while the parameters are unobserved (shaded white). 

$$ p(\mathbf{CO2}|\theta, \mathbf{t}) = \prod_{t=1}^{n} p(CO2_t|\theta , t) $$
$$ = \prod_{t=1}^{n} N(CO2_t | c_0 + c_1 t + c_2\cos{(2\pi (t + c_3))}, c_4^2) $$

, which is to say that the likelihood of observing a given datum of CO2 ($\text{CO2}_t$) at time index $t$ is Normally distributed, centered at a time-indexed linear-cosine function but varying with a small noise term. This is a bread-and-butter likelihood, easily encoding a practitioner's intuitions about the overall behavior of the CO2 data, while also allowing for uncertainty. As a notational shorthand, $ \theta = \{c_0, c_1, c_2, c_3, c_4\}$. Finally, while we can claim a fixed $n = 3242$, the total number of observations in the training set, the next portion of this project will focus on forecasting, which will extend the time series by 20 or so years to $n* = 3242 + 52*20 = 4,282$ observations. 

As such, the full Bayesian model has 5 parameters, each of which has a prior distribution informed by the initial exploratory analysis.

![](/Users/andrevacha/Desktop/cs146/Final Project/priors_lc.png)

1. Parameter $c_0$: Historical $\text{CO}_2$ Levels. 

    $$ p(c_0) \sim N(c_0 | 300, 20) $$

    This is naively the intercept term (seen earlier to be about 310 ppm), but it is more accurately described as the CO2 levels in the absence of long-term variability, ironically entailing the need for some variability. As such, we are constrained to a positive parameter, positioned around 300 ppm with some uncertainty in a Normal prior. The uncertainty ($\sigma ^2$) term is set to 20 as we know that 95% of this distribution will lie within [260, 340] ppm, which seems to be a reasonably constrained parameter space. 
    

2. Parameter $c_1$: Long-Term Linear Trend.

    $$ p(c_1) \sim N(c_1 | 0, 5) $$

    As a coefficent of the time-index, $c_1$ can be thought of as the slope: the change in CO2 each year * in the absence of seasonal and noise components*. This is also constrained to be positive, so 95% of our half-Normal will lie within [0, 10] ppm/year, which is a fairly large parameter space. We choose a normal because the probability density increases as we approach 0, and from the data exploration we know that the growth is around 1-2 ppm/year. Simultaneously, we don't want to underweigh the importance of the data with a sharp prior, so we let the distribution gradually lengthen out over the parameter space.

    During exploratory analysis, we observed indicatiosn that the trend, while positive, was not linear. The detrended data had a pronounced bow shape, suggesting an exponential or quadratic fit. In the interest of a benchmark, simpler model, we will use a linear trend.

3. Parameter $c_2$: Seasonal Amplitude. 

    $$ p(c_2) \sim N(c_2 | 0, 5) $$

    The amplitude defines the lower and upper extent of seasonal variation, which traces a sinusoidal pattern. The sinusoidal trace has two features. First, a negative amplitude is simply a reflection about the x-axis. Second, the phase space of its period (1 year) is explored within a [$-\pi$ , $\pi$] phase shift (paramter $c_3$). In other words, it is self-similar under constrained parameters. This is a boon when using MCMC methods for posterior sampling (as we soon will), as it signigicantly reduces the number of first-order moments the sampler typically fixates on (and samples poorly as a result). Following a coincidentally identical rationale above, we use a positive-constrained Normal centered at 0, with a standard deviation of 5. 

4. Parameter $c_3$: Phase Shift. 

    $$ p(c_3) \sim \text{Uniform}(c_3 | -\pi, \pi) $$

    In this and the subsequent model, we assume a fixed time period $T$ of 1 year. As a result, $\omega = \frac{2\pi}{T} = 2\pi $, leading to an overall phase space of [$-\pi$ , $\pi$]. We don't have any intuitions on the phase shift, so we use a flat distribution constrained to be in the range of [$-\pi$, $\pi$]. The author likes to call this a uniform *pi*er. (To detail why the entire phase space is explored within this constrained range, see Phase Plot in Appendix A)

5. Parameter $c_4$: Random Fluctuations away from the layered trend. 

    $$ p(c_4) \sim \text{InvGamma}(c_4 | 2.5, 2) $$

    Based on the distribution of noise marginals in the data analysis, we know that the noise is very fine-grained and less than one. An Inverse-Gamma distribution is appropriate for this parameter, with a shape parameter of 0 and a scale parameter of 0.1, as it distributions the majority of its mass within the range [0, 0.1] ppm. This is positively constrained and very sharp, as we know that noise plays a very modest role in the trend. 

## Exponential-Cosine Model

To rectify a key problem with the previous model, we swap a linear-long term trend with an exponential trend:

![Factor Graph](/Users/andrevacha/Desktop/cs146/Final Project/factor_graph_exp.png "Factor Graph")

The factorization is nearly identical to the previous model, except with an additional 6th parameter and transformed time representation. 

$$ p(\mathbf{CO2}|\theta, \mathbf{t}) = \prod_{t=1}^{n} p(CO2_t|\theta , t) $$

$$ = \prod_{t=1}^{n} N( CO2_t | c_0 + c_1 e^{c_2 t*} + c_3\cos{(2\pi (t + c_4))}, c_5^2) $$ 

![Priors Exp-Cosine Graph](/Users/andrevacha/Desktop/cs146/Final Project/priors_exp_cosine.png "Priors Exp-Cosine Graph")

While the rationale for shared parameters are the same, the new parameters deserve introduction:

1. Parameter $c_1$: Exponential Scale.

    $$ p(c_1) \sim N(c_1 | 25, 10) $$

    This simply rescales the exponential component. It is positively constrained, with a [5, 45] ppm probable range for the exponential scale, which is a good support.

2. Parameter $c_2$: Exponential Shape. 

    $$ p(c_2) \sim logN(c_2 | 1, 1) $$

    This parameter controls the tautness of the arc of the exponential component. Larger values of $c_2$ result in a more pronounced, taut arc, while smaller values result in a smoother arc. A lognormal with a mean of 1.5 and a standard deviation of 1 is a reasonable choice, as we know that a shallow arc is obtained from very small ($\leq 1$) values of $c_2$: the specified log-normal concentrates most of its mass in the range [0.5, 2.5], making this a good support for our prior knowledge. 
    
    Since the growth in $t$ is exponential, we need to apply an affine transformation to it to condition the outputs: we do this by applying `t/t.max()` to yield a fractional $t*$, such that it falls between [0, 1]. In other words, $e^60 = 1e26$ (!) is a computationally intractable number, while $e^1 = 2.71$ is much more feasible to compute.


# Inference

## Linear-Cosine Model
increases noise to compensate for the poor fit of the linear function to the long-term trend.
AME 
 

### MCMC Sampling
+ moderately informed priors + constraints on parameters = good sampling in inference. 
Pairplot of Parameter Samples
+ Sharp esimates! 
+ correlations
+ even samples (n_eff > 3000 for each parameter)
+ low autocorrelation
+ plot of phase space to avoid modal oversampling

## Exponential-Cosine Model
AME

### Test Statistics
to prove that it predicts data well is not to say that the model is "correct". 

# Predictions

## 40 Year Forecast


## 95% Confidence Intervals for 450 ppm Threshold


